In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [6]:
df_csv = pd.read_csv('Misdrijven_W_B.csv', sep=';')
df_xls = pd.read_excel('kwb-2018.xls')
df_csv = df_csv.rename(columns={'WijkenEnBuurten': 'gwb_code_10'})
df = df_xls.merge(df_csv, on = 'gwb_code_10', how='inner')
df.head()

,gwb_code_10,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,...,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad,ID,TotaalGeweldsEnSeksueleMisdrijven_22,Mishandeling_23,BedreigingEnStalking_24,OverigeGeweldsEnSeksueleMisdrijven_25
0,BU00030000,30000,Appingedam-Centrum,Appingedam,Buurt,BU00030000,1,2360,1120,1240,...,5,9901,1,3,1195,1552,25.0,17.0,5.0,3.0
1,BU00030001,30001,Appingedam-West,Appingedam,Buurt,BU00030001,1,3030,1505,1525,...,5,9903,5,4,896,1553,16.0,6.0,7.0,3.0
2,BU00030002,30002,Appingedam-Oost,Appingedam,Buurt,BU00030002,1,5820,2815,3005,...,11,9902,1,3,1119,1554,33.0,21.0,8.0,4.0
3,BU00030007,30007,Verspreide huizen Damsterdiep en Eemskanaal,Appingedam,Buurt,BU00030007,1,335,175,160,...,18,9903,4,5,348,1555,1.0,0.0,1.0,0.0
4,BU00030008,30008,Verspreide huizen ten zuiden van Eemskanaal,Appingedam,Buurt,BU00030008,1,100,55,45,...,28,9902,1,5,74,1556,1.0,1.0,0.0,0.0
